In [2]:
import pandas as pd
import numpy as np
import re
import warnings

# Cấu hình để hiển thị tất cả các cột của DataFrame
pd.set_option('display.max_columns', None)
# Bỏ qua các cảnh báo không quan trọng
warnings.filterwarnings('ignore')

print("Libraries imported successfully.")

Libraries imported successfully.


In [2]:
import requests

# Direct URL
url = "https://www.fns.usda.gov/sites/default/files/resource-files/usda-foods-database-sy2025-2026.xlsx"

# Nếu dùng VPN, không cần proxy. Nếu vẫn lỗi SSL, thêm verify=False (tạm thời, không an toàn lâu dài)
response = requests.get(url, verify=False)

if response.status_code == 200:
    with open("usda-foods-database-sy2025-2026.xlsx", "wb") as f:
        f.write(response.content)
    print("Tải thành công!")
else:
    print(f"Lỗi: {response.status_code} - Thử VPN hoặc proxy.")

Lỗi: 403 - Thử VPN hoặc proxy.


/Users/nguyenthuong/Documents/DietPlanning/env/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.fns.usda.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [5]:
# Tế bào 2: Định nghĩa đường dẫn và tải dữ liệu USDA
FOOD_DOT_COM_PATH = '/Users/nguyenthuong/Desktop/Food.com Recipes and Interactions/'
USDA_PATH = '/Users/nguyenthuong/Desktop/FoodData_Central_foundation_food_csv_2025-04-24/'


In [11]:
try:
    # Tải dữ liệu USDA
    food_df = pd.read_csv(USDA_PATH + 'food.csv')
    nutrient_df = pd.read_csv(USDA_PATH + 'nutrient.csv')
    food_nutrient_df = pd.read_csv(USDA_PATH + 'food_nutrient.csv')
    
    # Tải dữ liệu Food.com
    raw_recipes_df = pd.read_csv(FOOD_DOT_COM_PATH + 'RAW_recipes.csv')

    print("USDA data loaded successfully.")
except FileNotFoundError as e:
    print(f"Error loading USDA files: {e}")
    print("Please check your USDA_PATH.")

USDA data loaded successfully.


In [21]:
print(food_df.shape)
print(nutrient_df.shape)
print(food_nutrient_df.shape)
print(raw_recipes_df.shape)


(74175, 5)
(477, 5)
(155243, 11)
(231637, 12)


In [27]:
food_df.head()

,fdc_id,data_type,description,food_category_id,publication_date
0,319874,sample_food,"HUMMUS, SABRA CLASSIC",16.0,2019-04-01
1,319875,market_acquisition,"HUMMUS, SABRA CLASSIC",16.0,2019-04-01
2,319876,market_acquisition,"HUMMUS, SABRA CLASSIC",16.0,2019-04-01
3,319877,sub_sample_food,Hummus,16.0,2019-04-01
4,319878,sub_sample_food,Hummus,16.0,2019-04-01


In [28]:
nutrient_df.head()

,id,name,unit_name,nutrient_nbr,rank
0,2047,Energy (Atwater General Factors),KCAL,957.0,280.0
1,2048,Energy (Atwater Specific Factors),KCAL,958.0,290.0
2,1001,Solids,G,201.0,200.0
3,1002,Nitrogen,G,202.0,500.0
4,1003,Protein,G,203.0,600.0


In [29]:
food_nutrient_df.head()

,id,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,footnote,min_year_acquired
0,2201847,319877,1051,56.30,1.0,1.0,NaN,NaN,NaN,NaN,NaN
1,2201845,319877,1002,1.28,1.0,1.0,NaN,NaN,NaN,NaN,NaN
2,2201846,319877,1004,19.00,1.0,1.0,NaN,NaN,NaN,NaN,NaN
3,2201844,319877,1007,1.98,1.0,1.0,NaN,NaN,NaN,NaN,NaN
4,2201852,319878,1091,188.00,1.0,1.0,NaN,NaN,NaN,NaN,NaN


In [31]:
raw_recipes_df.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [32]:
raw_recipes_df.tail()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
231632,zydeco soup,486161,60,227978,2012-08-29,"['ham', '60-minutes-or-less', 'time-to-make', ...","[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",7,"['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",22
231633,zydeco spice mix,493372,5,1500678,2013-01-09,"['15-minutes-or-less', 'time-to-make', 'course...","[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1,['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",13
231634,zydeco ya ya deviled eggs,308080,40,37779,2008-06-07,"['60-minutes-or-less', 'time-to-make', 'course...","[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8
231635,cookies by design cookies on a stick,298512,29,506822,2008-04-15,"['30-minutes-or-less', 'time-to-make', 'course...","[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",9,['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",10
231636,cookies by design sugar shortbread cookies,298509,20,506822,2008-04-15,"['30-minutes-or-less', 'time-to-make', 'course...","[174.9, 14.0, 33.0, 4.0, 4.0, 11.0, 6.0]",5,"['whip sugar and shortening in a large bowl , ...","i've heard of the 'cookies by design' company,...","['granulated sugar', 'shortening', 'eggs', 'fl...",7


In [33]:
raw_recipes_df.sample(5)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
4873,american poverty pudding circa 1933,203790,35,335277,2007-01-07,"['60-minutes-or-less', 'time-to-make', 'course...","[406.5, 17.0, 109.0, 39.0, 26.0, 32.0, 21.0]",5,"['spread corn flakes in a greased , deep puddi...","lest we forget, a baked sweet pudding made wit...","['corn flakes', 'milk', 'sugar', 'ginger', 'eg...",8
170330,raita salad,291777,5,788546,2008-03-13,"['15-minutes-or-less', 'time-to-make', 'course...","[65.2, 2.0, 22.0, 1.0, 6.0, 5.0, 3.0]",1,['combine ingredient in a bowl and toss to coat'],a great side for indian dishes.,"['cucumber', 'tomatoes', 'red onions', 'plain ...",7
187474,simple sweet and mild curry blend,73674,5,41409,2003-10-20,"['15-minutes-or-less', 'time-to-make', 'course...","[263.7, 16.0, 7.0, 2.0, 18.0, 7.0, 15.0]",2,"['combine all ingrediendts , and store in a gl...",this curry blend is made from everyday ingredi...,"['ground coriander', 'turmeric', 'cumin', 'cin...",10
174537,river road cookbook sweet and sour pork,263410,30,394085,2007-11-05,"['30-minutes-or-less', 'time-to-make', 'course...","[475.7, 12.0, 248.0, 27.0, 54.0, 13.0, 24.0]",17,['beat the egg and incorporate the 2 tablespoo...,this is a wonderful--not sticky sweet!--sweet ...,"['egg', 'cornstarch', 'lean pork', 'oil', 'sug...",11
115182,jiffy corn bread mix tamale pie,420582,75,265679,2010-04-16,"['time-to-make', 'course', 'main-ingredient', ...","[636.9, 52.0, 14.0, 77.0, 65.0, 72.0, 16.0]",9,['mix cornbread according to package direction...,this recipe is from my starving student colleg...,"['jiffy cornbread mix', 'ground beef', 'taco s...",5


In [34]:
raw_recipes_df.describe()

,id,minutes,contributor_id,n_steps,n_ingredients
count,231637.000000,2.316370e+05,2.316370e+05,231637.000000,231637.000000
mean,222014.708984,9.398546e+03,5.534885e+06,9.765499,9.051153
std,141206.635626,4.461963e+06,9.979141e+07,5.995128,3.734796
min,38.000000,0.000000e+00,2.700000e+01,0.000000,1.000000
25%,99944.000000,2.000000e+01,5.690500e+04,6.000000,6.000000
50%,207249.000000,4.000000e+01,1.736140e+05,9.000000,9.000000
75%,333816.000000,6.500000e+01,3.982750e+05,12.000000,11.000000
max,537716.000000,2.147484e+09,2.002290e+09,145.000000,43.000000


In [36]:
raw_recipes_df['name'].describe()

count                             231636
unique                            230185
top       crock pot lemon garlic chicken
freq                                   3
Name: name, dtype: object

In [38]:
print(raw_recipes_df['name'].unique())

['arriba   baked winter squash mexican style'
 'a bit different  breakfast pizza' 'all in the kitchen  chili' ...
 'zydeco ya ya deviled eggs' 'cookies by design   cookies on a stick'
 'cookies by design   sugar shortbread cookies']


In [39]:
print(raw_recipes_df['name'].value_counts())

name
crock pot lemon garlic chicken                  3
gluten free chocolate chip cookies              3
chocolate peanut butter cookies                 3
three bean chili                                3
pop up rolls                                    3
                                               ..
easy pineapple cake                             1
easy pineapple chicken                          1
easy pineapple chili                            1
easy pineapple dessert                          1
cookies by design   sugar shortbread cookies    1
Name: count, Length: 230185, dtype: int64


count: Số lượng giá trị không rỗng.
unique: Số lượng giá trị duy nhất.
top: Giá trị xuất hiện nhiều nhất.
freq: Tần suất của giá trị top.

In [22]:
print(food_df.columns)
print(nutrient_df.columns)
print(food_nutrient_df.columns)
print(raw_recipes_df.columns)


Index(['fdc_id', 'data_type', 'description', 'food_category_id',
       'publication_date'],
      dtype='object')
Index(['id', 'name', 'unit_name', 'nutrient_nbr', 'rank'], dtype='object')
Index(['id', 'fdc_id', 'nutrient_id', 'amount', 'data_points', 'derivation_id',
       'min', 'max', 'median', 'footnote', 'min_year_acquired'],
      dtype='object')
Index(['name', 'id', 'minutes', 'contributor_id', 'submitted', 'tags',
       'nutrition', 'n_steps', 'steps', 'description', 'ingredients',
       'n_ingredients'],
      dtype='object')


In [17]:
food_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74175 entries, 0 to 74174
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   fdc_id            74175 non-null  int64  
 1   data_type         74175 non-null  object 
 2   description       74167 non-null  object 
 3   food_category_id  70444 non-null  float64
 4   publication_date  74175 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 2.8+ MB


In [18]:
nutrient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477 entries, 0 to 476
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            477 non-null    int64  
 1   name          477 non-null    object 
 2   unit_name     477 non-null    object 
 3   nutrient_nbr  465 non-null    float64
 4   rank          466 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 18.8+ KB


In [19]:
food_nutrient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155243 entries, 0 to 155242
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 155243 non-null  int64  
 1   fdc_id             155243 non-null  int64  
 2   nutrient_id        155243 non-null  int64  
 3   amount             155216 non-null  float64
 4   data_points        151745 non-null  float64
 5   derivation_id      154828 non-null  float64
 6   min                15516 non-null   float64
 7   max                15516 non-null   float64
 8   median             16118 non-null   float64
 9   footnote           22 non-null      object 
 10  min_year_acquired  30305 non-null   float64
dtypes: float64(7), int64(3), object(1)
memory usage: 13.0+ MB


In [23]:
raw_recipes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   name            231636 non-null  object
 1   id              231637 non-null  int64 
 2   minutes         231637 non-null  int64 
 3   contributor_id  231637 non-null  int64 
 4   submitted       231637 non-null  object
 5   tags            231637 non-null  object
 6   nutrition       231637 non-null  object
 7   n_steps         231637 non-null  int64 
 8   steps           231637 non-null  object
 9   description     226658 non-null  object
 10  ingredients     231637 non-null  object
 11  n_ingredients   231637 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 21.2+ MB


In [24]:
print(raw_recipes_df.isnull().sum())

name                 1
id                   0
minutes              0
contributor_id       0
submitted            0
tags                 0
nutrition            0
n_steps              0
steps                0
description       4979
ingredients          0
n_ingredients        0
dtype: int64


In [25]:
print(food_df.isnull().sum())

fdc_id                 0
data_type              0
description            8
food_category_id    3731
publication_date       0
dtype: int64


In [26]:
print(nutrient_df.isnull().sum())

id               0
name             0
unit_name        0
nutrient_nbr    12
rank            11
dtype: int64


In [30]:
print(food_nutrient_df.isnull().sum())

id                        0
fdc_id                    0
nutrient_id               0
amount                   27
data_points            3498
derivation_id           415
min                  139727
max                  139727
median               139125
footnote             155221
min_year_acquired    124938
dtype: int64


In [51]:
# Tế bào 3: Xác định các chất dinh dưỡng mục tiêu và xây dựng bảng tra cứu
# Định nghĩa các nutrient_id quan trọng dựa trên file nutrient.csv
# 1. Định nghĩa các nutrient_id chính xác dựa trên dữ liệu đã kiểm tra
print("--- Step 3.1: Defining target nutrient IDs ---")
NUTRIENT_IDS = {
    'calories': 1008,
    'protein_g': 1003,
    'fat_g': 1004,
    'carbs_g': 1005,
    'fiber_g': 1079
}

--- Step 3.1: Defining target nutrient IDs ---


In [52]:
# Verification
print(f"Target nutrients defined: {list(NUTRIENT_IDS.keys())}")
print("-" * 50)


Target nutrients defined: ['calories', 'protein_g', 'fat_g', 'carbs_g', 'fiber_g']
--------------------------------------------------


In [53]:
# Lọc bảng food_nutrient để chỉ giữ lại các chất dinh dưỡng cần thiết
# Sử dụng .isin() để lọc hiệu quả
print("--- Step 3.2: Filtering the main food_nutrient table ---")
# Lọc để chỉ giữ lại các hàng chứa các chất dinh dưỡng chúng ta quan tâm
food_nutrient_filtered_df = food_nutrient_df[food_nutrient_df['nutrient_id'].isin(NUTRIENT_IDS.values())].copy()
# Verification
print(f"Original food_nutrient_df shape: {food_nutrient_df.shape}")
print(f"Filtered food_nutrient_filtered_df shape: {food_nutrient_filtered_df.shape}")
print("Unique nutrient IDs in filtered data:", food_nutrient_filtered_df['nutrient_id'].unique())

--- Step 3.2: Filtering the main food_nutrient table ---
Original food_nutrient_df shape: (155243, 11)
Filtered food_nutrient_filtered_df shape: (8626, 11)
Unique nutrient IDs in filtered data: [1004 1079 1003 1008 1005]


In [54]:
print("First 5 rows of filtered data:")
display(food_nutrient_filtered_df.head())
print("-" * 50)

First 5 rows of filtered data:


,id,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,footnote,min_year_acquired
2,2201846,319877,1004,19.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
16,2201859,319882,1004,18.7,1.0,1.0,NaN,NaN,NaN,NaN,NaN
28,2201873,319892,1004,16.6,1.0,1.0,NaN,NaN,NaN,NaN,NaN
43,2201886,319899,1004,19.1,1.0,1.0,NaN,NaN,NaN,NaN,NaN
97,2201942,319908,1004,18.2,1.0,1.0,NaN,NaN,NaN,NaN,NaN


--------------------------------------------------


In [42]:
food_nutrient_filtered_df.sample(5)

,id,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,footnote,min_year_acquired
87202,8530804,749006,1004,34.1000,1.0,1.0,NaN,NaN,NaN,NaN,NaN
117046,27791046,2259475,1004,0.0000,1.0,1.0,NaN,NaN,NaN,NaN,NaN
133943,31224746,2515374,1005,36.2887,NaN,49.0,NaN,NaN,NaN,NaN,NaN
58972,2276302,335412,1004,1.7000,1.0,1.0,NaN,NaN,NaN,NaN,2016.0
97946,13338151,1105770,1079,2.1000,1.0,1.0,NaN,NaN,NaN,NaN,NaN


In [9]:
# Lấy tên chất dinh dưỡng từ nutrient_df để làm tên cột sau này
nutrient_id_to_name = nutrient_df[nutrient_df['id'].isin(NUTRIENT_IDS.values())].set_index('id')['name'].to_dict()

In [10]:

# Chuyển đổi tên cột từ ID sang tên dễ đọc
food_nutrient_filtered_df['nutrient_name'] = food_nutrient_filtered_df['nutrient_id'].map(nutrient_id_to_name)

In [55]:
print("--- Step 3.3: Creating a mapping from nutrient ID to the desired column name ---")
nutrient_id_to_col_name = {v: k for k, v in NUTRIENT_IDS.items()}

--- Step 3.3: Creating a mapping from nutrient ID to the desired column name ---


In [56]:
# Verification
print("Nutrient ID to Column Name Mapping:")
print(nutrient_id_to_col_name)
print("-" * 50)

Nutrient ID to Column Name Mapping:
{1008: 'calories', 1003: 'protein_g', 1004: 'fat_g', 1005: 'carbs_g', 1079: 'fiber_g'}
--------------------------------------------------


In [57]:

print("--- Step 3.4: Pivoting the nutrient data to transform rows into columns ---")
# Tái cấu trúc DataFrame: mỗi fdc_id là một hàng, mỗi chất dinh dưỡng là một cột
nutrient_pivot_df = food_nutrient_filtered_df.pivot_table(
    index='fdc_id',
    columns='nutrient_id',
    values='amount',
    aggfunc='first'
).rename(columns=nutrient_id_to_col_name)

--- Step 3.4: Pivoting the nutrient data to transform rows into columns ---


In [58]:
# Verification
print("Pivoting complete. This table will have NaN values initially.")
print(f"Shape of pivot table: {nutrient_pivot_df.shape}")
print("Sample of 5 random rows from the pivot table (before filling NaN):")
display(nutrient_pivot_df.sample(5))
print("-" * 50)

Pivoting complete. This table will have NaN values initially.
Shape of pivot table: (5801, 5)
Sample of 5 random rows from the pivot table (before filling NaN):


nutrient_id,protein_g,fat_g,carbs_g,calories,fiber_g
fdc_id,,,,,
2644289,7.8,1.256,21.03,NaN,NaN
335400,23.8,1.750,NaN,NaN,4.4
2515856,NaN,4.280,NaN,NaN,NaN
2004818,NaN,0.890,NaN,NaN,NaN
335382,25.7,1.680,NaN,NaN,4.4


--------------------------------------------------


In [59]:
# === Step 3.4: Chuyển đổi về DataFrame phẳng và đổi tên cột ===
print("\n--- Step 3.4: Flattening the pivot table and renaming columns ---")
# Reset index để chuyển fdc_id từ index thành cột thông thường
nutrient_flat_df = nutrient_pivot_df.reset_index()

# Đổi tên các cột nutrient ID thành tên dễ đọc
nutrient_flat_df.rename(columns=nutrient_id_to_col_name, inplace=True)
# Xóa tên của chỉ số cột (loại bỏ 'nutrient_id' bị dồn vào)
nutrient_flat_df.columns.name = None


--- Step 3.4: Flattening the pivot table and renaming columns ---


In [63]:
# Verification 3.4
print("\n--- Verification 3.4 ---")
print("DataFrame has been flattened. 'fdc_id' is now a regular column.")
print(f"Shape of flat table: {nutrient_flat_df.shape}")
print("First 5 rows of the flattened and renamed table:")
display(nutrient_flat_df.sample(5))
print("-" * 60)



--- Verification 3.4 ---
DataFrame has been flattened. 'fdc_id' is now a regular column.
Shape of flat table: (5801, 6)
First 5 rows of the flattened and renamed table:


,fdc_id,protein_g,fat_g,carbs_g,calories,fiber_g
4616,2515706,NaN,45.35,NaN,NaN,NaN
916,330465,26.5,NaN,NaN,NaN,NaN
3796,2263631,NaN,NaN,NaN,NaN,21.5
1847,335612,24.1,0.95,NaN,NaN,3.9
1272,333678,NaN,0.85,NaN,NaN,NaN


------------------------------------------------------------


In [64]:

print("--- Step 3.5: Merging nutritional data with food descriptions ---")
# Chỉ lấy các cột cần thiết từ food_df và loại bỏ các thực phẩm không có tên
food_descriptions_df = food_df[['fdc_id', 'description']].dropna(subset=['description'])
# Verification
print(f"Shape of food descriptions table: {food_descriptions_df.shape}")

# Hợp nhất hai DataFrame
# Dùng `reset_index()` để fdc_id trở thành một cột thông thường để merge
usda_lookup_df = pd.merge(
    food_descriptions_df,
    nutrient_pivot_df.reset_index(), # Quan trọng: reset_index() ở đây
    on='fdc_id',
    how='inner' # Chỉ giữ lại các thực phẩm có cả mô tả và ít nhất một giá trị dinh dưỡng
)


--- Step 3.5: Merging nutritional data with food descriptions ---
Shape of food descriptions table: (74167, 2)


In [65]:
# Verification
print(f"Shape of merged table (before cleaning): {usda_lookup_df.shape}")
print("Sample of 5 random rows from the merged table (still has NaN):")
display(usda_lookup_df.sample(5))
print("-" * 50)

Shape of merged table (before cleaning): (5801, 7)
Sample of 5 random rows from the merged table (still has NaN):


,fdc_id,description,protein_g,fat_g,carbs_g,calories,fiber_g
2684,790058,"FLOUR, WHOLE WHEAT (UNENRICHED)",NaN,NaN,NaN,NaN,11.40
3782,2263269,"peanut butter, creamy",NaN,NaN,NaN,NaN,6.16
986,330816,"Protein, ash,& moisture, Ground turkey, 93% le...",19.7000,NaN,NaN,NaN,NaN
33,320214,"MILK, 2%",NaN,1.68,NaN,NaN,NaN
3867,2346394,"Nuts, walnuts, English, halves, raw",14.5644,69.74,10.9096,NaN,5.21


--------------------------------------------------


In [66]:

# === Step 3.5: Hợp nhất với tên mô tả thực phẩm ===
print("\n--- Step 3.5: Merging with food descriptions ---")
# Loại bỏ các thực phẩm không có tên mô tả
food_descriptions_df = food_df.dropna(subset=['description'])

# Hợp nhất hai DataFrame
usda_lookup_df = pd.merge(
    food_descriptions_df,
    nutrient_flat_df,
    on='fdc_id',
    how='inner' # Chỉ giữ lại các thực phẩm có cả mô tả và ít nhất một giá trị dinh dưỡng
)


--- Step 3.5: Merging with food descriptions ---


In [67]:
# Verification 3.5
print("\n--- Verification 3.5 ---")
print(f"Shape before merge: food_descriptions_df={food_descriptions_df.shape}, nutrient_flat_df={nutrient_flat_df.shape}")
print(f"Shape of merged table (may still have NaNs): {usda_lookup_df.shape}")
print("A sample of 5 rows from the merged table:")
display(usda_lookup_df.sample(5, random_state=42))
print("-" * 60)


--- Verification 3.5 ---
Shape before merge: food_descriptions_df=(74167, 5), nutrient_flat_df=(5801, 6)
Shape of merged table (may still have NaNs): (5801, 10)
A sample of 5 rows from the merged table:


,fdc_id,data_type,description,food_category_id,publication_date,protein_g,fat_g,carbs_g,calories,fiber_g
3737,2262074,foundation_food,"Almond butter, creamy",12.0,2022-04-28,20.78734,53.04,21.23666,NaN,9.718
1103,332203,sub_sample_food,"TDF, Sauce, pasta, HUNTS original, canned spag...",6.0,2019-04-01,NaN,NaN,NaN,NaN,2.000
812,329383,sub_sample_food,"Whole eggs, dried",1.0,2019-04-01,NaN,37.70,NaN,NaN,NaN
1632,335397,agricultural_acquisition,"Beans, Dry, Navy, 213 (0% moisture)",16.0,2019-04-01,24.50000,1.43,NaN,NaN,4.600
1128,332384,sub_sample_food,"Lunchmeat, ham - NFY1210QA",7.0,2019-04-01,NaN,4.38,NaN,NaN,NaN


------------------------------------------------------------


In [68]:

# === Step 3.6: Dọn dẹp cuối cùng và xác thực toàn diện ===
print("\n--- Step 3.6: Final cleaning and comprehensive validation ---")
# Đổi tên cột 'description' để tránh xung đột
usda_lookup_df.rename(columns={'description': 'usda_description'}, inplace=True)
# Điền 0 cho các giá trị dinh dưỡng bị thiếu (NaN)
usda_lookup_df[list(NUTRIENT_IDS.keys())] = usda_lookup_df[list(NUTRIENT_IDS.keys())].fillna(0)


--- Step 3.6: Final cleaning and comprehensive validation ---


In [69]:

# === FINAL VALIDATION ===
print("\nUSDA Nutrient Lookup Dictionary created successfully.")
print("\n--- FINAL VALIDATION ---")
print(f"Shape of the final lookup table: {usda_lookup_df.shape}")
print("\nFirst 5 rows:")
display(usda_lookup_df.head())
print("\nData types and memory usage:")
usda_lookup_df.info()
print("\nChecking for any remaining null values:")
# is null().sum().sum() sẽ trả về tổng tất cả các giá trị null trong toàn bộ DataFrame
if usda_lookup_df.isnull().sum().sum() == 0:
    print("No null values found in the entire DataFrame.")
else:
    print("WARNING: Null values still exist. Please check the columns below:")
    print(usda_lookup_df.isnull().sum())
print("\nDescriptive statistics:")
display(usda_lookup_df.describe())


USDA Nutrient Lookup Dictionary created successfully.

--- FINAL VALIDATION ---
Shape of the final lookup table: (5801, 10)

First 5 rows:


,fdc_id,data_type,usda_description,food_category_id,publication_date,protein_g,fat_g,carbs_g,calories,fiber_g
0,319877,sub_sample_food,Hummus,16.0,2019-04-01,0.0,19.0,0.0,0.0,0.0
1,319882,sub_sample_food,Hummus,16.0,2019-04-01,0.0,18.7,0.0,0.0,0.0
2,319892,sub_sample_food,Hummus,16.0,2019-04-01,0.0,16.6,0.0,0.0,0.0
3,319899,sub_sample_food,Hummus,16.0,2019-04-01,0.0,19.1,0.0,0.0,0.0
4,319908,sub_sample_food,Hummus,16.0,2019-04-01,0.0,18.2,0.0,0.0,0.0



Data types and memory usage:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5801 entries, 0 to 5800
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   fdc_id            5801 non-null   int64  
 1   data_type         5801 non-null   object 
 2   usda_description  5801 non-null   object 
 3   food_category_id  5721 non-null   float64
 4   publication_date  5801 non-null   object 
 5   protein_g         5801 non-null   float64
 6   fat_g             5801 non-null   float64
 7   carbs_g           5801 non-null   float64
 8   calories          5801 non-null   float64
 9   fiber_g           5801 non-null   float64
dtypes: float64(6), int64(1), object(3)
memory usage: 453.3+ KB

Checking for any remaining null values:
fdc_id               0
data_type            0
usda_description     0
food_category_id    80
publication_date     0
protein_g            0
fat_g                0
carbs_g              0
calories  

,fdc_id,food_category_id,protein_g,fat_g,carbs_g,calories,fiber_g
count,5.801000e+03,5721.000000,5801.000000,5801.000000,5801.000000,5801.000000,5801.000000
mean,1.409249e+06,11.362524,4.700509,6.838500,1.136719,4.517540,1.588982
std,1.019070e+06,5.657412,9.455341,14.182486,7.591438,38.608438,3.157871
min,3.198770e+05,1.000000,0.000000,0.000000,-0.705000,0.000000,0.000000
25%,3.350920e+05,9.000000,0.000000,0.100000,0.000000,0.000000,0.000000
50%,1.105430e+06,11.000000,0.000000,1.070000,0.000000,0.000000,0.000000
75%,2.514517e+06,16.000000,0.000000,3.830000,0.000000,0.000000,2.830000
max,2.730715e+06,25.000000,79.900000,99.400000,99.600000,833.000000,43.940000


In [74]:
# === Step 3.7: Tổng hợp Dữ liệu để tạo Bảng tra cứu Cuối cùng ===
print("\n--- Step 3.7: Aggregating data to create a representative nutrient profile ---")
print(f"Shape before aggregation: {usda_lookup_df.shape}")


--- Step 3.7: Aggregating data to create a representative nutrient profile ---
Shape before aggregation: (5801, 10)


In [75]:

# --- Verification (Pre-Aggregation) ---
print("\nData types of columns before aggregation:")
print(usda_lookup_df.info()) # In ra kiểu dữ liệu để thấy rõ các cột 'object'


Data types of columns before aggregation:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5801 entries, 0 to 5800
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   fdc_id            5801 non-null   int64  
 1   data_type         5801 non-null   object 
 2   usda_description  5801 non-null   object 
 3   food_category_id  5721 non-null   float64
 4   publication_date  5801 non-null   object 
 5   protein_g         5801 non-null   float64
 6   fat_g             5801 non-null   float64
 7   carbs_g           5801 non-null   float64
 8   calories          5801 non-null   float64
 9   fiber_g           5801 non-null   float64
dtypes: float64(6), int64(1), object(3)
memory usage: 453.3+ KB
None


In [82]:

# --- Implementation (Corrected) ---
# 1. Xác định các cột số mà chúng ta muốn tính trung bình
numeric_cols_to_agg = ['calories', 'protein_g', 'fat_g', 'carbs_g', 'fiber_g']

In [83]:

# 2. Tạo cột chữ thường để nhóm
usda_lookup_df['usda_description_lower'] = usda_lookup_df['usda_description'].str.lower()

In [84]:

# 3. Thực hiện groupby
# Chỉ chọn các cột cần thiết (cột nhóm + các cột số) trước khi tính trung bình
usda_aggregated_lookup_df = usda_lookup_df[['usda_description_lower'] + numeric_cols_to_agg].groupby(
    'usda_description_lower', 
    as_index=False
).mean()

In [79]:

# --- FINAL VALIDATION (trên bảng đã tổng hợp) ---
print("\n✅ Final Aggregated USDA Nutrient Lookup Dictionary created successfully.")
print("\n--- FINAL VALIDATION ---")
print(f"Shape of the final aggregated lookup table: {usda_aggregated_lookup_df.shape}")


✅ Final Aggregated USDA Nutrient Lookup Dictionary created successfully.

--- FINAL VALIDATION ---
Shape of the final aggregated lookup table: (2385, 6)


In [85]:

print("\nVerifying 'Hummus' nutrient profile (should have non-zero values for protein, carbs):")
# Sử dụng .str.contains để tìm 'hummus'
hummus_profile = usda_aggregated_lookup_df[usda_aggregated_lookup_df['usda_description_lower'].str.contains('hummus', na=False)]
if not hummus_profile.empty:
    display(hummus_profile)
else:
    print("No 'hummus' profile found in the aggregated data.")

print("\nFirst 5 rows of the final aggregated table:")
display(usda_aggregated_lookup_df.head())

print("\nChecking for any remaining null values:")
if usda_aggregated_lookup_df.isnull().sum().sum() == 0:
    print("No null values found in the aggregated DataFrame.")
else:
    print("WARNING: Null values still exist. Please check the aggregation logic.")
    print(usda_aggregated_lookup_df.isnull().sum())


Verifying 'Hummus' nutrient profile (should have non-zero values for protein, carbs):


,usda_description_lower,calories,protein_g,fat_g,carbs_g,fiber_g
1277,hummus,0.0,0.00,17.136364,0.0,0.0
1278,hummus - nfy1213zx,0.0,0.00,0.000000,0.0,5.5
1279,hummus - nfy12140q,0.0,0.00,0.000000,0.0,5.7
1280,hummus - nfy12141f,0.0,0.00,0.000000,0.0,5.4
1281,hummus - nfy12141z,0.0,0.00,0.000000,0.0,5.5
1282,hummus - nfy12142o,0.0,0.00,0.000000,0.0,4.9
1283,hummus - nfy12143e,0.0,0.00,0.000000,0.0,5.7
1284,"hummus, commercial",229.0,7.35,17.100000,14.9,5.4



First 5 rows of the final aggregated table:


,usda_description_lower,calories,protein_g,fat_g,carbs_g,fiber_g
0,"almond butter, creamy",0.0,1.222785,28.081765,1.249215,5.144882
1,"almond milk, refrigerated, unsweetened",0.0,0.000000,0.778750,0.000000,0.000000
2,"almond milk, unsweetened, plain, refrigerated",0.0,0.656250,1.558000,0.670750,0.000000
3,"almond milk, unsweetened, plain, shelf stable",0.0,0.032629,0.646529,0.019830,0.000000
4,almonds dry roasted salted,0.0,0.000000,57.746154,0.000000,0.000000



Checking for any remaining null values:
No null values found in the aggregated DataFrame.


In [ ]:
# Tế bào 4: Tải và lấy mẫu dữ liệu công thức
try:
    raw_recipes_df = pd.read_csv(FOOD_DOT_COM_PATH + 'RAW_recipes.csv')
    print("Food.com recipes loaded successfully.")
    
    # Lấy mẫu ngẫu nhiên 20,000 công thức để xử lý
    # random_state=42 để đảm bảo kết quả có thể tái tạo được
    recipes_sample_df = raw_recipes_df.sample(n=20000, random_state=42)
    print(f"Sampled {len(recipes_sample_df)} recipes for processing.")

except FileNotFoundError as e:
    print(f"Error loading Food.com file: {e}")
    print("Please check your FOOD_DOT_COM_PATH.")

In [81]:
print("\nVerifying 'Hummus' nutrient profile (should have non-zero values for protein, carbs):")
# Sử dụng .str.contains để tìm 'hummus'
hummus_profile = usda_aggregated_lookup_df[usda_aggregated_lookup_df['usda_description_lower'].str.contains('hummus', na=False)]
if not hummus_profile.empty:
    display(hummus_profile)
else:
    print("No 'hummus' profile found in the aggregated data.")

print("\nFirst 5 rows of the final aggregated table:")
display(usda_aggregated_lookup_df.head())

print("\nChecking for any remaining null values:")
if usda_aggregated_lookup_df.isnull().sum().sum() == 0:
    print("✔️ No null values found in the aggregated DataFrame.")
else:
    print("WARNING: Null values still exist. Please check the aggregation logic.")
    print(usda_aggregated_lookup_df.isnull().sum())



Verifying 'Hummus' nutrient profile (should have non-zero values for protein, carbs):


,usda_description_lower,calories,protein_g,fat_g,carbs_g,fiber_g
1277,hummus,0.0,0.00,17.136364,0.0,0.0
1278,hummus - nfy1213zx,0.0,0.00,0.000000,0.0,5.5
1279,hummus - nfy12140q,0.0,0.00,0.000000,0.0,5.7
1280,hummus - nfy12141f,0.0,0.00,0.000000,0.0,5.4
1281,hummus - nfy12141z,0.0,0.00,0.000000,0.0,5.5
1282,hummus - nfy12142o,0.0,0.00,0.000000,0.0,4.9
1283,hummus - nfy12143e,0.0,0.00,0.000000,0.0,5.7
1284,"hummus, commercial",229.0,7.35,17.100000,14.9,5.4



First 5 rows of the final aggregated table:


,usda_description_lower,calories,protein_g,fat_g,carbs_g,fiber_g
0,"almond butter, creamy",0.0,1.222785,28.081765,1.249215,5.144882
1,"almond milk, refrigerated, unsweetened",0.0,0.000000,0.778750,0.000000,0.000000
2,"almond milk, unsweetened, plain, refrigerated",0.0,0.656250,1.558000,0.670750,0.000000
3,"almond milk, unsweetened, plain, shelf stable",0.0,0.032629,0.646529,0.019830,0.000000
4,almonds dry roasted salted,0.0,0.000000,57.746154,0.000000,0.000000



Checking for any remaining null values:
✔️ No null values found in the aggregated DataFrame.


In [ ]:
# Tế bào 5: Xây dựng các hàm xử lý
# Hàm 1: Chuẩn hóa tên một thành phần
def normalize_ingredient(ing_name):
    """Chuẩn hóa chuỗi tên thành phần để tìm kiếm."""
    ing_name = ing_name.lower()
    # Loại bỏ số lượng và các đơn vị phổ biến
    ing_name = re.sub(r'(\d+/\d+|\d+\s\d+/\d+|\d+)\s*(oz|ounce|ounces|g|gram|grams|kg|lb|lbs|cup|cups|teaspoon|teaspoons|tablespoon|tablespoons)\b', '', ing_name)
    # Loại bỏ các từ mô tả phổ biến
    ing_name = re.sub(r'\b(diced|chopped|fresh|minced|sliced|cooked|frozen|canned|peeled|seeded)\b', '', ing_name)
    # Loại bỏ các ký tự đặc biệt, giữ lại chỉ chữ cái và khoảng trắng
    ing_name = re.sub(r'[^a-z\s]', '', ing_name)
    # Loại bỏ khoảng trắng thừa
    ing_name = ' '.join(ing_name.split())
    return ing_name.strip()

# Hàm 2: Hàm chính để xử lý một công thức (một hàng trong DataFrame)
def process_recipe_row(row, lookup_df):
    """
    Tính toán tổng dinh dưỡng cho một công thức.
    Giả định mỗi thành phần có khối lượng 100g.
    """
    try:
        # eval() có thể không an toàn, nhưng với dữ liệu này thì chấp nhận được
        ingredients_list = eval(row['ingredients'])
    except:
        return None # Bỏ qua nếu định dạng 'ingredients' bị lỗi

    total_nutrition = {
        'calories': 0.0, 'protein_g': 0.0, 'fat_g': 0.0, 
        'carbs_g': 0.0, 'fiber_g': 0.0
    }
    matched_ingredients = 0

    for ing in ingredients_list:
        clean_ing = normalize_ingredient(ing)
        if not clean_ing: # Bỏ qua nếu sau khi chuẩn hóa thành chuỗi rỗng
            continue
            
        # Tìm kiếm trong từ điển USDA. Đây là bước tốn thời gian nhất.
        # Chúng ta tìm một sự trùng khớp chính xác trước để tăng tốc độ.
        exact_match = lookup_df[lookup_df['usda_description'].str.lower() == clean_ing]
        
        if not exact_match.empty:
            match = exact_match.iloc[0]
        else:
            # Nếu không có, tìm kiếm chuỗi con
            partial_matches = lookup_df[lookup_df['usda_description'].str.contains(clean_ing, case=False, na=False)]
            if not partial_matches.empty:
                # Lấy kết quả đầu tiên (một sự đơn giản hóa)
                match = partial_matches.iloc[0]
            else:
                match = None
        
        if match is not None:
            matched_ingredients += 1
            # Giả định 100g cho mỗi thành phần
            total_nutrition['calories'] += match['calories']
            total_nutrition['protein_g'] += match['protein_g']
            total_nutrition['fat_g'] += match['fat_g']
            total_nutrition['carbs_g'] += match['carbs_g']
            total_nutrition['fiber_g'] += match['fiber_g']

    # Một công thức được coi là hợp lệ nếu tính được calo và có ít nhất 1 thành phần được khớp
    if total_nutrition['calories'] > 50 and matched_ingredients > 0:
        return {
            'id': row['id'],
            'name': row['name'],
            'description': row['description'],
            'steps': row['steps'],
            'ingredients_list': ingredients_list,
            **total_nutrition
        }
    return None

print("Processing functions are ready.")

In [ ]:
# Tế bào 6: Áp dụng các hàm và tạo DataFrame sạch
# Kích hoạt thanh tiến trình cho pandas.apply
tqdm.pandas(desc="Processing Recipes")

# Áp dụng hàm xử lý trên từng hàng
# axis=1 đảm bảo hàm được áp dụng cho từng hàng
processed_data_list = recipes_sample_df.progress_apply(process_recipe_row, axis=1, lookup_df=usda_lookup_df)

# Lọc bỏ các kết quả None (công thức không hợp lệ) và chuyển thành DataFrame
clean_recipes_df = pd.DataFrame(processed_data_list.dropna().tolist())

print(f"\nProcessing complete. Found {len(clean_recipes_df)} valid recipes.")
clean_recipes_df.head()

In [ ]:
# Tế bào 7: Thiết lập kết nối Django và nạp dữ liệu
import os
import sys
import django
from sqlalchemy import create_engine

# --- Thiết lập môi trường Django Standalone ---
# Thêm đường dẫn dự án vào sys.path
# !!! QUAN TRỌNG: Thay thế bằng đường dẫn thực tế đến thư mục DietPlanning !!!
project_path = '/Users/nguyenthuong/Documents/DietPlanning' 
if project_path not in sys.path:
    sys.path.append(project_path)

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'core.settings')
try:
    django.setup()
    print("Django environment setup successful.")
except Exception as e:
    print(f"Error setting up Django environment: {e}")

# --- Bắt đầu nạp dữ liệu ---
from planner.models import Recipe
from django.conf import settings

# Lấy thông tin database từ file settings.py
db_settings = settings.DATABASES['default']
db_url = f"postgresql://{db_settings['USER']}:{db_settings['PASSWORD']}@{db_settings['HOST']}:{db_settings['PORT']}/{db_settings['NAME']}"

try:
    engine = create_engine(db_url)
    print("Database engine created successfully.")
    
    # Xóa dữ liệu cũ trong bảng Recipe trước khi nạp mới (để tránh trùng lặp)
    print("Clearing old data from 'planner_recipe' table...")
    with engine.connect() as connection:
        connection.execute(Recipe.objects.all().delete())
    print("Old data cleared.")

    # Đảm bảo các cột DataFrame khớp với tên field trong model Recipe
    df_to_load = clean_recipes_df.copy()
    # Chú ý cột 'steps' và 'ingredients_list' cần là JSON
    # Pandas to_sql không xử lý JSONField tốt, chúng ta sẽ nạp qua ORM
    
    print(f"Preparing to load {len(df_to_load)} recipes into the database...")
    
    # Sử dụng bulk_create của Django để hiệu quả hơn
    recipes_to_create = [
        Recipe(
            # Chú ý: Cột id trong df có thể không khớp với id tự tăng của DB
            # Chúng ta sẽ không set id để DB tự quản lý
            name=row['name'],
            description=row['description'],
            steps=eval(row['steps']) if isinstance(row['steps'], str) else row['steps'],
            ingredients_list=row['ingredients_list'],
            calories=row['calories'],
            protein_g=row['protein_g'],
            fat_g=row['fat_g'],
            carbs_g=row['carbs_g'],
            fiber_g=row['fiber_g']
        )
        for index, row in tqdm(df_to_load.iterrows(), total=df_to_load.shape[0], desc="Creating Recipe Objects")
    ]
    
    Recipe.objects.bulk_create(recipes_to_create, batch_size=500) # batch_size để tối ưu bộ nhớ
    
    print("\nData loaded into PostgreSQL successfully!")

except Exception as e:
    print(f"\nAn error occurred during data loading: {e}")